In [ ]:
import rasterio
from rasterio.warp import transform as warp_transform
#https://planetarycomputer.microsoft.com/dataset/io-lulc-9-class#Example-Notebook
import dask.array as da
import dask.distributed
from matplotlib.colors import ListedColormap
import pystac_client
import pyproj
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import planetary_computer
import rasterio
import rasterio.features
import stackstac
from pystac.extensions.eo import EOExtension as eo
import dask.dataframe as dd
import datetime
import odc.stac
import rioxarray
import time

In [ ]:
# for DEM
import rioxarray
from pystac_client import Client
import planetary_computer
from concurrent.futures import ThreadPoolExecutor
# Coordinates for Fort Collins, CO

min_lon = -109.0448
max_lon = -102.0415
min_lat = 36.9924
max_lat = 41.0037

# Define the bounding box of interest
bbox_of_interest = [min_lon, min_lat, max_lon, max_lat]  # Replace with your actual bounding box coordinates

# Create a catalog search for the specified collection and bounding box
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(collections=["cop-dem-glo-30"], bbox=bbox_of_interest)
items = list(search.get_items())
print(f"Returned {len(items)} items")

def process_item(item, idx):
    print(f"Processing item {idx+1}/{len(items)}")
    signed_asset = planetary_computer.sign(item.assets["data"])
    data = (
        rioxarray.open_rasterio(signed_asset.href)
        .squeeze()
        .drop("band")
        # Uncomment to reduce data size
        # .coarsen({"y": 5, "x": 5})
        # .mean()
    )
    data.rio.write_crs("EPSG:4326", inplace=True)
    
    output_tif_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\DEMTIF\\output_dataDEM_{idx}.tif"
    data.rio.to_raster(output_tif_path)

# Use ThreadPoolExecutor to process items in parallel
# with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your CPU
#     for i, item in enumerate(items):
#         executor.submit(process_item, item, i)
# Use ThreadPoolExecutor to process only the last three items in parallel
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your CPU
    for i, item in enumerate(items[-3:]):  # Change this line to process only the last three items
        executor.submit(process_item, item, len(items) - 3 + i)  # Adjust index for correct labeling

print("All files have been processed and saved.")
print("All files have been processed and saved.")


In [ ]:
### for DEM
### WORKS WITH NO BOARDER!
import os
import glob
import subprocess

#tif_dir = r"D:\GlobalPCL\lidarHAG"
#output_tif = r"D:\GlobalPCL\lidarHAG\LIDAR_GEDI_CO.tif"
tif_dir = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\DEMTIF"
output_tif  = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\DEMTIF\DEM_CO.tif"


tifs = glob.glob(os.path.join(tif_dir, "*dataDEM*.tif"))

# Prepare the gdal_merge command for HAG
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    #"-ot", "Byte",
    "-o", output_tif,
    "-n", "-9999",
    "-a_nodata","-9999",
    
    
] + tifs

# Run the gdal_merge command for HAG and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Check if the command for HAG was successful
if process_hag.returncode != 0:
    # An error occurred, print the error
    print("Error occurred while merging TIFF files HAG:")
    print(process_hag.stderr)
else:
    print("TIFF files merged successfully for HAG.")



In [ ]:
# FOR DEM
from osgeo import gdal, ogr, osr

# Define the paths to your rasters
lidarHAG_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\DEMTIF\DEM_CO.tif"
pcl_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(lidarHAG_path, gdal.GA_ReadOnly)
target_ds = gdal.Open(pcl_path, gdal.GA_ReadOnly)


# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_DEM_CO.tif"
out_ds = driver.Create(output_path, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

print("Resampling completed. Output saved at:", output_path)

# Define the paths to your rasters
lidarHAG_path = output_path

# Open the datasets
lidarHAG_ds = gdal.Open(lidarHAG_path, gdal.GA_ReadOnly)
pcl_ds = gdal.Open(pcl_path, gdal.GA_ReadOnly)

# Function to extract raster details
def get_raster_details(dataset):
    geotransform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    pixel_width = geotransform[1]
    pixel_height = geotransform[5]
    
    return {
        "cols": cols,
        "rows": rows,
        "pixel_width": pixel_width,
        "pixel_height": pixel_height,
        "projection": projection
    }

# Get details for both datasets
lidarHAG_details = get_raster_details(lidarHAG_ds)
pcl_details = get_raster_details(pcl_ds)

# Print the details
print("LIDAR HAG Raster Details:")
print("Columns:", lidarHAG_details['cols'])
print("Rows:", lidarHAG_details['rows'])
print("Pixel Width:", lidarHAG_details['pixel_width'])
print("Pixel Height:", lidarHAG_details['pixel_height'])
print("Projection:", lidarHAG_details['projection'])

print("\nPCL Raster Details:")
print("Columns:", pcl_details['cols'])
print("Rows:", pcl_details['rows'])
print("Pixel Width:", pcl_details['pixel_width'])
print("Pixel Height:", pcl_details['pixel_height'])
print("Projection:", pcl_details['projection'])

# Close datasets
lidarHAG_ds = None
pcl_ds = None
print("DONE!!!")

In [ ]:
# Coordinates for Fort Collins, CO

min_lon = -109.0448
max_lon = -102.0415
min_lat = 36.9924
max_lat = 41.0037

# Define the bounding box
bbox_of_interest = [min_lon, min_lat, max_lon, max_lat]

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
search = catalog.search(
    collections=["cop-dem-glo-30"],
    bbox=bbox_of_interest
)
items = list(search.get_items())
print(f"Returned {len(items)} items")

In [ ]:
#
#i = 0

for i in range(len(items)):
    i = 45
    print(i)
    signed_asset = planetary_computer.sign(items[i].assets["data"])
    data = (
        rioxarray.open_rasterio(signed_asset.href)
        .squeeze()
        .drop("band")
        #.coarsen({"y": 5, "x": 5})
        #.mean()
    )
    
    data.rio.write_crs("EPSG:4326", inplace=True)  # Change "EPSG:4326" to the appropriate CRS if different
    
    # Specify the path where you want to save the TIFF file
    output_tif_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\DEMTIF\\output_dataDEM_{i}.tif"
    
    # Save the data as a GeoTIFF file
    data.rio.to_raster(output_tif_path)

In [ ]:
central_lon = (min_lon+max_lon)/2

time_of_interest = "2021-08-01/2021-08-31"

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)

items = search.item_collection()

# least_cloudy_item = min(items, key=lambda item: eo.ext(item).cloud_cover)

# utm_crs = pyproj.CRS.from_user_input(f'EPSG:326{(int((central_lon + 180) / 6) % 60) + 1}')

# # Create a stack from the least cloudy item
# if least_cloudy_item:
#     sentinel_stack = stackstac.stack(
#         [least_cloudy_item],
#         bounds_latlon=bbox_of_interest,
#         epsg=utm_crs.to_epsg(),
#         resolution=10,
#         assets=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"],  
#         fill_value=0
#     )

#     print("Stack created successfully from the least cloudy item")
# else:
#     print("No least cloudy item found to create a stack")

In [ ]:
items

In [ ]:
# len(items)
# i=1
# signed_asset = planetary_computer.sign(items[i].assets["data"])
# data = (
#     rioxarray.open_rasterio(signed_asset.href)
#     .squeeze()
#     .drop("band")
#     #.coarsen({"y": 5, "x": 5})
#     #.mean()
# )

# data.rio.write_crs("EPSG:4326", inplace=True)  # Change "EPSG:4326" to the appropriate CRS if different

# # Specify the path where you want to save the TIFF file
# output_tif_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\DEMTIF\\output_dataSENT_{i}.tif"

# # Save the data as a GeoTIFF file
# data.rio.to_raster(output_tif_path)

In [ ]:
import rioxarray

# Assume 'stacked' is your DataArray from the previous steps

# Print dimensions to understand the structure
print("DataArray dimensions before adjustment:", stacked.dims)

# If there are more than three dimensions, you may need to squeeze out singleton dimensions
# or select specific indices if you know the dimensions to keep
stacked = stacked.squeeze()  # This removes any singleton dimensions if they exist

# Double-check the dimensions after squeezing
print("DataArray dimensions after squeezing:", stacked.dims)

# Ensure the array is in the order (band, y, x) if there are three dimensions
if len(stacked.dims) == 3:
    stacked = stacked.transpose('band', 'y', 'x')  # Adjust this based on your actual dimension names

# Now try saving the raster again
output_tif_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\DEMTIF\\output_dataSENT_{i}.tif"
stacked.rio.to_raster(output_tif_path)

print(f"GeoTIFF saved successfully at {output_tif_path}.")


In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt

#north, south, east, west = 40.6369, 40.5531, -105.0049, -105.0763  
#north, south, east, west = 41.0023, 40.5328, -104.9826, -105.6780
north, south, east, west = 41.0037, 36.9924, -102.0415, -109.0448
#north, south, east, west = 41.0037, 36.9924, -102.0415, -104.0448

# Define the custom filter for waterways
#custom_filter = '["waterway"~"river|streams|canals"]'
custom_filter = '["waterway"~"river"]'


#ox.config(overpass_settings='[out:json][timeout:180][max_query_area_size:100000]')

# Get the waterway network for the specified bounding box using the custom filter
#graph = ox.graph_from_bbox(north, south, east, west, custom_filter=custom_filter, simplify=True, retain_all=True, truncate_by_edge=True)
bbox = (north, south, east, west)  # your current north, south, east, west values
graph = ox.graph_from_bbox(bbox=bbox, custom_filter=custom_filter, simplify=True, retain_all=True, truncate_by_edge=True)


gdf = ox.graph_to_gdfs(graph, nodes=False)

# Convert the graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(graph)

# Plot the waterway network
fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=1, edgecolor='blue')
#plt.title('Waterways in Fort Collins')
plt.show()






import rasterio
from rasterio.features import rasterize
from rasterio.windows import from_bounds
import numpy as np
import scipy.ndimage

# Define the decay constant
lambda_decay = 0.03  # This value might need adjustment based on your specific use case

# Assuming 'gdf' is your correctly located GeoDataFrame
# Define the bounding box coordinates
# north, south, east, west = 40.7369, 40.5531, -105.0149, -105.0763

in_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\Input\\pcl_west_wgs_CO_3.tif'
# Load the reference TIFF to use for transformations and metadata
with rasterio.open(in_path) as src:
    meta = src.meta.copy()

    # Convert geographic coordinates to pixel coordinates
    window = from_bounds(west, south, east, north, src.transform)
    transform = rasterio.windows.transform(window, src.transform)

    # Correct the window dimensions to integers
    window_width = int(window.width)
    window_height = int(window.height)

    # Create an empty raster array for the size of the window
    raster = np.zeros((window_height, window_width), dtype=np.uint8)

    # Rasterize directly into the window shape
    shapes = ((geom, 1) for geom in gdf['geometry'])
    burned = rasterize(shapes, out=raster, fill=0, transform=transform, all_touched=True)

    # Compute the Euclidean distance from each cell to the nearest non-zero cell
    distance_grid = scipy.ndimage.distance_transform_edt(burned == 0)

    # Apply exponential decay to the distance grid
    decay_grid = np.exp(-lambda_decay * distance_grid)

    # Prepare metadata for saving the decay grid
    clipped_meta = meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": window_height,
        "width": window_width,
        "transform": transform,
        "dtype": rasterio.float32,
        "count": 1,
        "compress": 'lzw'
    })

    # Save the decay grid to a new TIFF file
    output_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\Input\\exponential_decay_CO_river3.tif'
    with rasterio.open(output_path, 'w', **clipped_meta) as dst:
        dst.write(decay_grid.astype(np.float32), 1)






from osgeo import gdal, ogr, osr

# Define the paths to your rasters
lidarHAG_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\exponential_decay_CO_river.tif"
pcl_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(lidarHAG_path, gdal.GA_ReadOnly)
target_ds = gdal.Open(pcl_path, gdal.GA_ReadOnly)


# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_RIVERS_CO.tif"
out_ds = driver.Create(output_path, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

print("Resampling completed. Output saved at:", output_path)

# Define the paths to your rasters
lidarHAG_path = output_path

# Open the datasets
lidarHAG_ds = gdal.Open(lidarHAG_path, gdal.GA_ReadOnly)
pcl_ds = gdal.Open(pcl_path, gdal.GA_ReadOnly)

# Function to extract raster details
def get_raster_details(dataset):
    geotransform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    pixel_width = geotransform[1]
    pixel_height = geotransform[5]
    
    return {
        "cols": cols,
        "rows": rows,
        "pixel_width": pixel_width,
        "pixel_height": pixel_height,
        "projection": projection
    }

# Get details for both datasets
lidarHAG_details = get_raster_details(lidarHAG_ds)
pcl_details = get_raster_details(pcl_ds)

# Print the details
print("LIDAR HAG Raster Details:")
print("Columns:", lidarHAG_details['cols'])
print("Rows:", lidarHAG_details['rows'])
print("Pixel Width:", lidarHAG_details['pixel_width'])
print("Pixel Height:", lidarHAG_details['pixel_height'])
print("Projection:", lidarHAG_details['projection'])

print("\nPCL Raster Details:")
print("Columns:", pcl_details['cols'])
print("Rows:", pcl_details['rows'])
print("Pixel Width:", pcl_details['pixel_width'])
print("Pixel Height:", pcl_details['pixel_height'])
print("Projection:", pcl_details['projection'])

# Close datasets
lidarHAG_ds = None
pcl_ds = None
print("DONE!!!")


In [5]:
import osmnx as ox
import matplotlib.pyplot as plt



# # Define the bounding box coordinates (south, north, west, east) for Fort Collins
# #north, south, east, west = 40.6369, 40.5531, -105.0049, -105.0763  # Adjust these coordinates as needed
# #north, south, east, west = 40.7369, 40.5531, -105.0149, -105.0763
# north, south, east, west = 41.0037, 36.9924, -102.0415, -109.0448


# # Get the road network for the specified bounding box and network type ('drive', 'walk', 'bike')
# graph = ox.graph_from_bbox(north, south, east, west, network_type='drive', simplify=True)


# gdf = ox.graph_to_gdfs(graph, nodes=False)


# # Plot the road network
# ox.plot_graph(graph)
# plt.show()





import rasterio
from rasterio.features import rasterize
from rasterio.windows import from_bounds
import numpy as np
import scipy.ndimage

# Define the decay constant
lambda_decay = 0.03  # This value might need adjustment based on your specific use case

# Assuming 'gdf' is your correctly located GeoDataFrame
# Define the bounding box coordinates
# north, south, east, west = 40.7369, 40.5531, -105.0149, -105.0763

in_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\Input\\pcl_west_wgs_CO_3.tif'
# Load the reference TIFF to use for transformations and metadata
with rasterio.open(in_path) as src:
    meta = src.meta.copy()

    # Convert geographic coordinates to pixel coordinates
    window = from_bounds(west, south, east, north, src.transform)
    transform = rasterio.windows.transform(window, src.transform)

    # Correct the window dimensions to integers
    window_width = int(window.width)
    window_height = int(window.height)

    # Create an empty raster array for the size of the window
    raster = np.zeros((window_height, window_width), dtype=np.uint8)

    # Rasterize directly into the window shape
    shapes = ((geom, 1) for geom in gdf['geometry'])
    burned = rasterize(shapes, out=raster, fill=0, transform=transform, all_touched=True)

    # Compute the Euclidean distance from each cell to the nearest non-zero cell
    distance_grid = scipy.ndimage.distance_transform_edt(burned == 0)

    # Apply exponential decay to the distance grid
    decay_grid = np.exp(-lambda_decay * distance_grid)

    # Prepare metadata for saving the decay grid
    clipped_meta = meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": window_height,
        "width": window_width,
        "transform": transform,
        "dtype": rasterio.float32,
        "count": 1,
        "compress": 'lzw'
    })

    # Save the decay grid to a new TIFF file
    output_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\Input\\exponential_decay_CO_ROADS.tif'
    with rasterio.open(output_path, 'w', **clipped_meta) as dst:
        dst.write(decay_grid.astype(np.float32), 1)






from osgeo import gdal, ogr, osr

# Define the paths to your rasters
lidarHAG_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\exponential_decay_CO_ROADS.tif"
pcl_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(lidarHAG_path, gdal.GA_ReadOnly)
target_ds = gdal.Open(pcl_path, gdal.GA_ReadOnly)


# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_ROADS_CO.tif"
out_ds = driver.Create(output_path, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

print("Resampling completed. Output saved at:", output_path)

# Define the paths to your rasters
lidarHAG_path = output_path

# Open the datasets
lidarHAG_ds = gdal.Open(lidarHAG_path, gdal.GA_ReadOnly)
pcl_ds = gdal.Open(pcl_path, gdal.GA_ReadOnly)

# Function to extract raster details
def get_raster_details(dataset):
    geotransform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    pixel_width = geotransform[1]
    pixel_height = geotransform[5]
    
    return {
        "cols": cols,
        "rows": rows,
        "pixel_width": pixel_width,
        "pixel_height": pixel_height,
        "projection": projection
    }

# Get details for both datasets
lidarHAG_details = get_raster_details(lidarHAG_ds)
pcl_details = get_raster_details(pcl_ds)

# Print the details
print("LIDAR HAG Raster Details:")
print("Columns:", lidarHAG_details['cols'])
print("Rows:", lidarHAG_details['rows'])
print("Pixel Width:", lidarHAG_details['pixel_width'])
print("Pixel Height:", lidarHAG_details['pixel_height'])
print("Projection:", lidarHAG_details['projection'])

print("\nPCL Raster Details:")
print("Columns:", pcl_details['cols'])
print("Rows:", pcl_details['rows'])
print("Pixel Width:", pcl_details['pixel_width'])
print("Pixel Height:", pcl_details['pixel_height'])
print("Projection:", pcl_details['projection'])

# Close datasets
lidarHAG_ds = None
pcl_ds = None
print("DONE!!!")


Resampling completed. Output saved at: C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_ROADS_CO.tif
LIDAR HAG Raster Details:
Columns: 7154
Rows: 4743
Pixel Width: 0.000285385769080235
Pixel Height: -0.00030922507779886164
Projection: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]

PCL Raster Details:
Columns: 7154
Rows: 4743
Pixel Width: 0.000285385769080235
Pixel Height: -0.00030922507779886164
Projection: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
DONE!!!
